In [1]:
%load_ext autoreload
%autoreload 2
import os
#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ["PMIX_MCA_gds"]="hash"

# Import useful packages
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
from qiskit_metal.toolbox_metal import math_and_overrides
from qiskit_metal.qlibrary.core import QComponent
from collections import OrderedDict

# To create plots after geting solution data.
import matplotlib.pyplot as plt
import numpy as np

# Packages for the simple design
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

# Set up chip design as planar, multiplanar also available
design = designs.DesignPlanar({}, overwrite_enabled=True)

# Set up chip dimensions 
design.chips.main.size.size_x = '4.6mm'
design.chips.main.size.size_y = '2.4mm'
design.chips.main.size.size_z = '-280um'
design.chips.main.size.center_x = '0mm'
design.chips.main.size.center_y = '-1mm'

# Resonator and feedline gap width (W) and center conductor width (S) are set to 50 Ohm 
design.variables['cpw_width'] = '10 um' #S
design.variables['cpw_gap'] = '6 um' #W 


# Create GUI
gui = MetalGUI(design)

# Lauchpad 1
x1 = '-2mm'
y1 = '0mm'
launch_options1 = dict(chip='main', pos_x=x1, pos_y=y1, orientation='360', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP1 = LaunchpadWirebond(design, 'LP1', options = launch_options1)

# Launchpad 2
x2 = '2mm'
y1 = '0mm'
launch_options2 = dict(chip='main', pos_x=x2, pos_y=y1, orientation='180', lead_length='30um', pad_height='103um', 
                      pad_width='103um', pad_gap='60um')
LP2 = LaunchpadWirebond(design, 'LP2', options = launch_options2)

# Using path finder to connect the two launchpads
TL = RoutePathfinder(design, 'TL', options = dict(chip='main', trace_width ='10um',
                                            trace_gap ='6um',
                                            fillet='90um',                                                            
                                            hfss_wire_bonds = True,
                                            lead=dict(end_straight='0.1mm'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='LP1',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='LP2',
                                                    pin='tie')
                                            )))


######################
# lambda/4 resonator1#
######################
otg1 = OpenToGround(design, 'otg1', options=dict(chip='main', pos_x='-0.2mm',  pos_y='-40um', orientation = 180))
stg1 = ShortToGround(design, 'stg1', options=dict(chip='main', pos_x='0mm',  pos_y='-1.35mm', orientation = -90))

# Use RouteMeander to fix the total length of the resonator
res1 = RouteMeander(design, 'resonator1',  Dict(
        trace_width ='10um',
        trace_gap ='6um',
        total_length='3.7mm',
        hfss_wire_bonds = False,
        fillet='99.9 um',
        lead = dict(start_straight='300um'),
        pin_inputs=Dict(
        start_pin=Dict(component='otg1', pin='open'),
        end_pin=Dict(component='stg1', pin='short')), ))

# rebuild the GUI
gui.rebuild()

# Capacitance Simulation

In [2]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
cap_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "solns_to_save": 2,                                            #number of electromagnetic field visualizations to save
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 160e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 6e-3,                                             #minimum element size for the mesh in mm
            "mesh_sampling": 120,                                          #number of points to mesh along a geometry
            "sim_directory": 'C:\\PALACE_Simulations\\'                    #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '20',
            "sim_time": '3:00:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Capacitance', 'cap_single_res', design, cap_user_options, ports = [], hpc_options = HPC_options)
sim.run_simulation()

Creating geometry for Capacitance Simulation.
No ports for processing.
metal cap phys group [1, 2, 3]
metal cap names ['metal_0', 'metal_1', 'metal_2']
Geometry successfully built in Gmsh.
metal 1
metal 2
metal 3
Finely meshing surfaces: [5, 6]
Mesh successfully built in Gmsh.
Directory 'cap_single_res' created at 'C:\PALACE_Simulations\cap_single_res'.
Simulation files created.


# Eigenmode Simulation

In [ ]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
eigen_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "starting_freq": 7.5,                                          #starting frequency in GHz 
            "number_of_freqs": 3,                                          #number of eigenmodes to find
            "solns_to_save": 2,                                            #number of electromagnetic field visualizations to save
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 170e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 6e-3,                                             #minimum element size for the mesh in mm
            "mesh_sampling": 150,                                          #number of points to mesh along a geometry
            "sim_directory": 'C:\\PALACE_Simulations\\'                    #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '25',
            "sim_time": '3:30:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Eigenmode', 'eigen_single_res', design, eigen_user_options, ports = ['LP1','LP2'], hpc_options = HPC_options)
sim.run_simulation()

# Driven Simulation

In [ ]:
from PALACE_Simulation import PALACE_Simulation

#Eigenmode Simulation Options
driven_user_options = {
            "mesh_refinement":  0,                                         #refines mesh in PALACE - essetially divides every mesh element in half
            "dielectric_material": "silicon",                              #choose dielectric material - 'silicon' or 'sapphire'
            "start_freq": 8.06,                                            #start frequency in GHz 
            "stop_freq": 8.08,                                             #stop frequency in GHz 
            "freq_step": 0.0001,                                           #size of frequency step                                                        
            "save_step": 50,                                               #number of steps before saving an electromagnetic field visualization
            "solver_order": 2,                                             #increasing solver order increases accuracy of simulation, but significantly increases sim time
            "solver_tol": 1.0e-8,                                          #error residual tolerance foriterative solver
            "solver_maxits": 100,                                          #number of solver iterations
            "mesh_max": 170e-3,                                            #maxiumum element size for the mesh in mm
            "mesh_min": 6e-3,                                              #minimum element size for the mesh in mm
            "mesh_sampling": 150,                                          #number of points to mesh along a geometry
            "sim_directory": r'C:/PALACE_Simulations/'                     #parent directory where all simulation files will be saved
}

HPC_options = {
            "hpc_nodes": '3',
            "cpus_per_node": '20',
            "sim_time": '3:00:00',
            "sim_memory": '250G',
            "account_name": 'a_fedorov',
            "palace_location": '/scratch/project/palace-sqdlab/Palace-Project-NEW/palace/build_v0.12/bin/palace-x86_64.bin',
            "input_files_location": '/scratch/project/palace-sqdlab/inputFiles/',
            "output_files_location": '/scratch/project/palace-sqdlab/outputFiles/'
}

sim = PALACE_Simulation('Driven', 'driven_single_res', design, driven_user_options, ports = ['LP1','LP2'], hpc_options = HPC_options)
sim.run_simulation()

In [3]:
import gmsh
#gmsh.initialize()
gmsh.fltk.run()